# ウェブデータ処理１: ウェブスクレイピング


ある特定のサイトからハイパーリンクが張られているページの情報をすべてダウンロードしたい場合や、   
頻繁に更新されるサイトの情報を定期的にダウンロードしたい場合は、   
それを自動で行うプログラムを作成し、利用するのが効率的です。   
Web上の様々なサイトの情報を自動的に収集することをクロール（それを行うプログラムをクローラ）、   
また収集した情報を利用しやすい形に処理することまでを含めてスクレイピングと呼びます。

ここでは、簡単なウェブスクレイピングについて学びたいと思います。   

なお、以下では実際に様々なWebサイトにアクセスします。   
**whileなどのループで短時間で同じサイトにアクセスしまくるのは絶対にやめましょう。**

1回きりのアクセスであれば、そのサイトをブラウザで開くのと同じですが、
例えばwhile文でループさせるなどして短時間内に大量のサイトにアクセスしてしまうと、
DoS攻撃（Denial of Service attack：Webサーバやそれにつながる通信網に意図的に過剰な負荷をかけることで、
サービス提供を阻害すること）とみなされて、そのサイトへのアクセスを遮断されます
（それ以前に、そのサイトに迷惑をかけることになりますね）。   

クロールする際はループで同じ同じサーバにアクセスすることになりますが、そうする場合は、十分な時間間隔（サーバの規模にもよりますが30秒以上。小規模なサーバだとリソースも少ないので、それでも遮断されることがあります）をあけてアクセスするようにしてください。   
そうすると、当然一定期間内に集められる情報が少なくなりますが、そういうものだとあきらめてください。。。

## 1. Webページの取得

まず、URLを指定してWebページの情報を取得します。  
ここでは`requests`というモジュールを使用します。これはAnacondaではデフォルトでインストールされるパッケージです。   
アクセス先は東京大学のホームページとします。

In [1]:
import random
import requests

url = 'https://www.u-tokyo.ac.jp/ja/index.html'

res = requests.get(url)
res.encoding = res.apparent_encoding  # 文字コードを設定
print(res.text)

<!DOCTYPE html>
<html lang="ja">
    <head>
        <meta charset="UTF-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">
        <title>東京大学</title>
        <meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=2,minimum-scale=1" user-scalable="yes">
        <meta name="description" content="東京大学のオフィシャルサイトです。大学案内、学部・大学院等の紹介、研究活動・国際活動、入学案内等、東京大学の情報をご覧いただけます。">
        <meta name="keywords" content="">
        <meta name="copyright" content="(C)東京大学">

        <link rel="shortcut icon" href="/content/100074614.ico">

        <meta property="og:title" content="東京大学">
        <meta property="og:site_name" content="東京大学">
        <meta property="og:type" content="website">
        <meta property="og:description" content="東京大学のオフィシャルサイトです。大学案内、学部・大学院等の紹介、研究活動・国際活動、入学案内等、東京大学の情報をご覧いただけます。">
        <meta property="og:url" content="https://www.u-tokyo.ac.jp/ja/index.html">
        <meta property="og:image" content="https://www.u-tokyo.ac.jp/content/400

## 2. HTMLの解析

WebページはHTML (HyperText Markup Language）という記述形式で書かれています。   
HTML形式では、
```
<a href="あるサイトのURL">あるサイトの名前</a>
```
とすることで、「あるサイトの名前」という文に「あるサイトのURL」のリンクを張ったり、

```
<font size="6" color="#ff6600">あいうえお</font>
```
とすることで、「あいうえお」のフォントサイズを「6」、フォントの色を「#ff6600」に指定することを指しています。   
このコードをブラウザで見ると、ブラウザがそのコードに従って、リンクを張ったり文字の大きさや色を変えたりしてくれるわけです。

ここで、HTMLでは、基本的に`<何かのタグ>`と`</先のタグと同じタグ>`で挟まれるのに注意してください。   
例えば上の例では`<a>`と`</a>`で挟まれていたり、`<font>`と`</font>`で挟まれていますよね。   
これらの対のタグは、それらで挟まれた文字列に対してパラメータを指定するものです。   
以降では簡単のため、「始まりのタグ」や「終わりのタグ」と呼ぶことにしましょう。   

なお、これらは入れ子にすることができます。   
例えば

```
<a href="http://aaa.www">こんなサイトは<font size="18">ありません！</font></a>
```
のようにすれば、「こんなさいとはありません！」にリンクが張られ、「ありません！」のフォントのサイズが「18」に設定されます。 

実はこのjupyter notebookもHTMLで書かれています。   
ですから、ここにHTMLのタグを書き込むと、ブラウザにその指示が伝わって以下のようになります。

<a href="http://aaa.www">こんなサイトは<font size="18">ありません！</font></a>

## 2.1 HTMLパーザ
さて、ブラウザで読み込んで使用する場合はこれでいいのですが、我々がこのデータを使う際には、フォントのサイズやハイパーリンク先のアドレスといったタグを含むコードは使いづらいですよね。   
そこで、このようなHTMLソースを使いやすいデータ形式に整理しましょう。   
これをパースと呼び、パースしてくれるプログラムをパーザと呼びます。   
ここではHTMLのコードをパースするので、「HTMLパーザ」を使います。

ここではシンプルなHTMLパーザとして、Python標準モジュールである`html.parser`を使用します。  
パーザにはほかにも`html5lib`や`lxml`などがあります。   
また、パースした結果を使いやすくするモジュールとして、デフォルトでインストールされている
`Beautiful soup`というモジュールを使用します。


In [2]:
from bs4 import BeautifulSoup

# res.textには、上のセルでダウンロードしたHTMLソースが代入されています
soup = BeautifulSoup(res.text, "lxml")


## 2.2 パージング結果の利用

これで、`soup`にはHTMLソースをパースした結果が代入されました。   
それではこの中身を見ていきたいと思います。   

HTMLでは、そのページ全体のタイトルを、`<title>題名</title>`というように`title`という名前のタグで指定します（これは、ブラウザの上のタブのところに表示されるテキストです）。   
`soup.title`とすると、`title`のタグがつけられた文のうち、一番最初にヒットしたものを出力します（`title`は通常、HTMLソースに一度しか現れません）。

In [3]:
print(soup.title)

<title>東京大学</title>


タグの名前`name`や内部のテキスト`text`は以下のようにして取り出せます。

In [4]:
print(soup.title.name)
print(soup.title.text)

title
東京大学


## 2.3 何度も現れるタグの処理

`title`はソースに1度しか現れないので上のコードでいいですが、`<a href="http://aaa.com>リンクはこちら</a>`のようなハイパーリンク情報などは、同じページに何度も現れますよね。   
このような情報を取り出すには`find_all`という関数を利用します。

ハイパーリンクの書式は、   
`<a href="[リンク先のURL]">リンクを張る文字</a>`   
です。   
この`[リンク先のURL]`を取り出してみましょう。   
まずはハイパーリンクを示す`a`のタグが貼られている部分を抽出します。

In [5]:
soup.find_all('a')

[<a href="/ja/index.html"><img alt="東京大学" class="u-img-responsive" src="/content/400109180.jpg"/></a>,
 <a href="#content">本文へ</a>,
 <a href="/ja/about/campus-guide/index.html">アクセス・キャンパスマップ</a>,
 <a href="/ja/general/contact.html">お問い合わせ</a>,
 <a href="http://utf.u-tokyo.ac.jp/index.html" target="_blank">寄附をお考えの方<img alt="別ウィンドウで開く" src="/content/100075725.png"/></a>,
 <a class="is-open" href="#"><span lang="en">Language</span></a>,
 <a href="/en/index.html"><span lang="en">English</span></a>,
 <a href="/zh/index.html"><span lang="zh">中文</span></a>,
 <a href="/ko/index.html"><span lang="ko">한국어</span></a>,
 <a href="/ja/index.html"><span lang="en">HOME</span></a>,
 <a class="l-gnav__focus" href="/focus/ja/index.html"><img alt="UTokyo FOCUS" src="/content/100074616.jpg"/></a>,
 <a href="/ja/about/index.html">大学案内</a>,
 <a href="/ja/about/index.html#category1">大学概要</a>,
 <a href="/ja/about/index.html#category2">キャンパス案内</a>,
 <a href="/ja/about/president/b01.html">総長室から</a>,
 <a href="/j

リンク先のURLは、`a`のタグの中にある`href`に代入されている値ですから、リンク先のURLを出力したいときは、以下のようにすればいいでしょう。

In [6]:
for address in soup.find_all('a'):
    print('ハイパーリンクが張られている文:', address.text)
    print('URL: ', address['href'])

ハイパーリンクが張られている文: 
URL:  /ja/index.html
ハイパーリンクが張られている文: 本文へ
URL:  #content
ハイパーリンクが張られている文: アクセス・キャンパスマップ
URL:  /ja/about/campus-guide/index.html
ハイパーリンクが張られている文: お問い合わせ
URL:  /ja/general/contact.html
ハイパーリンクが張られている文: 寄附をお考えの方
URL:  http://utf.u-tokyo.ac.jp/index.html
ハイパーリンクが張られている文: Language
URL:  #
ハイパーリンクが張られている文: English
URL:  /en/index.html
ハイパーリンクが張られている文: 中文
URL:  /zh/index.html
ハイパーリンクが張られている文: 한국어
URL:  /ko/index.html
ハイパーリンクが張られている文: HOME
URL:  /ja/index.html
ハイパーリンクが張られている文: 
URL:  /focus/ja/index.html
ハイパーリンクが張られている文: 大学案内
URL:  /ja/about/index.html
ハイパーリンクが張られている文: 大学概要
URL:  /ja/about/index.html#category1
ハイパーリンクが張られている文: キャンパス案内
URL:  /ja/about/index.html#category2
ハイパーリンクが張られている文: 総長室から
URL:  /ja/about/president/b01.html
ハイパーリンクが張られている文: 称号授与
URL:  /ja/about/index.html#category4
ハイパーリンクが張られている文: 東京大学の歴史
URL:  /ja/about/index.html#category5
ハイパーリンクが張られている文: 広報・刊行物
URL:  /ja/about/index.html#category6
ハイパーリンクが張られている文: 情報公開
URL:  /ja/about/index.html#category7
ハイパーリンクが張られ

## 2.4 find_allに複数条件の指定
`find_all`では複数の条件を指定することもできます。   
例えば、東京大学のホームページのニュースコーナー「UTokyo FOCUS」は次のような書式で書かれています。

```
<div class="p-slider-pc">
    <div><a href="/focus/ja/features/voices069.html"><img src="/content/400119832.jpg" alt="">
        <div class="p-slider-msg">
            <div class="p-slider-msg-inner">UTOKYO VOICES<br>光合成研究から次世代太陽電池の開発へ。光エネルギー変換にかけた35年。</div>
        </div>
    </a></div>
    <div><a href="/opendays/index.html"><img src="/content/400119155.jpg" alt="">
        <div class="p-slider-msg">
            <div class="p-slider-msg-inner">「高校生のためのオープンキャンパス2019」<br>2019年8月7日(水)･ 8日(木)に実施します。</div>
        </div>
    </a></div>
</div>
```

つまり、タグが`div`、クラスが`p-slider-pc`でマークアップされたものの中の、   
タグが`div`、クラスが`p-slider-msg-inner`でマークアップされたテキストが一つの「UTokyo FOCUS」ということになります。   
よって、「UTokyo FOCUS」のテキスト一覧は以下のようにして取得することができます。


In [7]:
for div1 in soup.find_all('div', attrs={"class": "p-slider-pc"}):
    for div2 in div1.find_all('div', attrs={"class": "p-slider-msg"}):
        print(div2.text.replace('\n', ''))

# こちらでもOK
#for div in soup.select(".p-slider-pc"):
#    for div2 in div.select('.p-slider-msg'): 
#        print(div2.text.replace('\n', ''))


FSIプロジェクトシリーズ国連の持続可能な開発目標（SDGs）に貢献する学内の研究活動を紹介していきます。
UTOKYO VOICES光合成研究から次世代太陽電池の開発へ。光エネルギー変換にかけた35年。
「高校生のためのオープンキャンパス2019」2019年8月7日(水)･ 8日(木)に実施します。
UTokyo FOCUS足立区と東大の｢おいしい｣関係
キミの東大薬学研究は病気のメカニズム解明や創薬だけでなく、基礎生物学にも大きな影響を及ぼせる―薬学部・富田泰輔教授
東京大学広報誌「淡青」38号
東京カレッジ世界の研究者・知識人と新たな知を拓き、伝える。東京カレッジが始まります。


## 2.5 画像のクロール

画像の場合は、`<img src="画像へのリンク", alt="画像が表示できなかった場合の代替テキスト">`となっているため、
このページに貼られている画像のリストは以下のようにして取得できます。   

In [8]:
for address in soup.find_all('img'):
    print('URL: ', address['src'])
    print('代替テキスト: ', address['alt'])

URL:  /content/400109180.jpg
代替テキスト:  東京大学
URL:  /content/100075725.png
代替テキスト:  別ウィンドウで開く
URL:  /content/100074616.jpg
代替テキスト:  UTokyo FOCUS
URL:  /content/100116151.png
代替テキスト:  大学案内
URL:  /content/100116153.png
代替テキスト:  学部・大学院等
URL:  /content/100116109.png
代替テキスト:  入学案内
URL:  /content/100116116.png
代替テキスト:  教育・学生生活
URL:  /content/100116161.png
代替テキスト:  研究活動
URL:  /content/100116132.png
代替テキスト:  社会連携
URL:  /content/100116137.png
代替テキスト:  産学連携
URL:  /content/100116078.png
代替テキスト:  国際交流
URL:  /content/100116096.png
代替テキスト:  卒業生
URL:  /content/100074653.png
代替テキスト:  別ウィンドウで開く
URL:  /content/400119944.jpg
代替テキスト:  
URL:  /content/400119832.jpg
代替テキスト:  
URL:  /content/400119155.jpg
代替テキスト:  
URL:  /content/400118046.jpg
代替テキスト:  
URL:  /content/400117191.jpg
代替テキスト:  別ウィンドウで開く
URL:  /content/400110613.jpg
代替テキスト:  
URL:  /content/400109797.jpg
代替テキスト:  別ウィンドウで開く
URL:  /content/400119944.jpg
代替テキスト:  
URL:  /content/400119832.jpg
代替テキスト:  
URL:  /content/400119155.jpg
代替テキスト:  
URL:  /con

なお、画像へのリンクは、多くの場合、絶対アドレスではなく相対アドレスで指定されていることに注意してください。   
画像のリンク先に置かれている画像をとってきたい場合は、いまアクセスしているURLを相対アドレスの前につなげる必要があります。

In [9]:
# `img`タグの情報を一つだけとってきます
address = soup.find('img')
print('画像ファイルのアドレス: ', address['src'])

# もしその'src'に'http'という文字列が含まれていないならば、
# それは相対アドレスなので、このページのURLを前につなげます
if 'http' not in address['src']:
    abs_address = 'https://www.u-tokyo.ac.jp/' + address['src']    
    print('画像ファイルの絶対アドレス: ', abs_address)
    
    # この教材の冒頭で使用したrequetsを使って画像ファイルをダウンロードしましょう
    r = requests.get(abs_address, stream=True)
    
    # 無事ダウンロードに成功したら、'img'という名前のフォルダの下に保存しましょう
    if r.status_code == 200:
        fname = 'img/' + abs_address.split('/')[-1]
        print('画像の保存先: ', fname)
        with open(fname, 'wb') as f:
            f.write(r.content)    
    else:
        print('画像の保存に失敗しました。')


画像ファイルのアドレス:  /content/400109180.jpg
画像ファイルの絶対アドレス:  https://www.u-tokyo.ac.jp//content/400109180.jpg
画像の保存先:  img/400109180.jpg


ダウンロードした画像は、この教材がおかれているフォルダの中にある`img`という名前のフォルダの中に置かれているはずです。   
確認してみてください。

HTMLでは他にも様々なタグが使われます。   
タグの種類や意味は[ここ](http://www.htmq.com/html/indexm.shtml)を参照してください。   
なお、HTMLソースのタグは大文字・小文字を区別しません。

Beautiful Soupにはほかにも様々なパラメータが用意されています。   
詳しくは[こちらのドキュメント](http://kondou.com/BS4/)を参照してください。